In [1]:
import warnings
warnings.filterwarnings('ignore')

# Research Question 1

## Does the style differ between different authors and projects?

a) Authors: See evaluation below

b) Projects: Not (yet) implemented [Dataloaders and Pairs are built up using authors only]

In [2]:
import pandas as pd
import numpy as np
import torch
import pytorch_lightning as L
from tqdm import tqdm

In [3]:
# Load Style_Model from trained model

import sys
sys.path.append('..')

from util.style_model import StyleModel

model = StyleModel()
model.load_state_dict(torch.load('../model/Style_Model.pt'))

<All keys matched successfully>

In [4]:
# load testing data
test_data = pd.read_pickle('../data/04c_Test_Set.pkl')

# use all data as test data to compare with baselines (very slow)
# test_data = pd.read_pickle('../data/03_Subset_Frequent_Committers.pkl')

### Calculate Embedding Centroid for each author

In [5]:
authors_centroids = {}

for group in tqdm(test_data.groupby('author_email')):
    author_embeddings = np.array([model(message).squeeze().detach().numpy() for message in group[1]['message']])
    authors_centroids[group[0]] = np.mean(author_embeddings, axis = 0)

100%|██████████| 7/7 [02:44<00:00, 23.46s/it]


### Calculate Cosine Distance between Author Centroids

In [6]:
from sklearn.metrics.pairwise import euclidean_distances

centroids_array = [value for value in authors_centroids.values()]

distances = pd.DataFrame(euclidean_distances(centroids_array, centroids_array))

for i in range(len(distances)):
    for j in range(i + 1):
        distances[j][i] = 0

distances.style.background_gradient(cmap='coolwarm').set_precision(3)

,0,1,2,3,4,5,6
0,0.000,0.288,0.245,0.525,0.227,0.357,0.306
1,0.000,0.000,0.304,0.521,0.306,0.388,0.321
2,0.000,0.000,0.000,0.515,0.192,0.260,0.234
3,0.000,0.000,0.000,0.000,0.525,0.566,0.513
4,0.000,0.000,0.000,0.000,0.000,0.285,0.211
5,0.000,0.000,0.000,0.000,0.000,0.000,0.199
6,0.000,0.000,0.000,0.000,0.000,0.000,0.000


### Half-Automated Evaluation of Style by K-Means Summary

Check K-Means Summary for Number of Authors and Projects per Cluster. If the number is low, it indicates that styles can be assigned to the different authors.

In [7]:
from sklearn.cluster import KMeans

embeddings = [model(message).squeeze().detach().numpy() for message in tqdm(test_data['message'])]

clustering = KMeans(100)
clustering.fit(embeddings)
clustering_prediction = clustering.predict(embeddings)

100%|██████████| 10684/10684 [02:49<00:00, 63.18it/s]


In [8]:
import sys
sys.path.append('..')

from util.clustering_eval import clustering_summary

k_means_summary = clustering_summary(clustering_prediction, clustering.n_clusters, test_data)
k_means_summary

There are 7 different authors.
There are 91 different projects.


,Number of Messages,Number of different Authors,Average number of commits per different Author,Most common Author,Number of different Projects,Average number of commits per different Project,Most common project
0,103.0,7.0,14.714286,"('mark@mark-story.com', 50)",21.0,4.904762,"('cakephp_cakephp', 47)"
1,88.0,7.0,12.571429,"('marijnh@gmail.com', 63)",15.0,5.866667,"('codemirror_CodeMirror', 50)"
2,96.0,6.0,16.000000,"('mark@mark-story.com', 42)",18.0,5.333333,"('cakephp_cakephp', 41)"
3,119.0,7.0,17.000000,"('pedro@algarvio.me', 49)",14.0,8.500000,"('saltstack_salt', 69)"
4,123.0,7.0,17.571429,"('marijnh@gmail.com', 41)",21.0,5.857143,"('saltstack_salt', 42)"
...,...,...,...,...,...,...,...
95,46.0,4.0,11.500000,"('thatch45@gmail.com', 28)",3.0,15.333333,"('saltstack_salt', 40)"
96,29.0,4.0,7.250000,"('marijnh@gmail.com', 22)",5.0,5.800000,"('codemirror_CodeMirror', 22)"
97,54.0,7.0,7.714286,"('mark@mark-story.com', 19)",19.0,2.842105,"('cakephp_cakephp', 17)"
98,81.0,4.0,20.250000,"('pedro@algarvio.me', 33)",8.0,10.125000,"('saltstack_salt', 53)"


In [9]:
k_means_summary.describe()

,Number of Messages,Number of different Authors,Average number of commits per different Author,Number of different Projects,Average number of commits per different Project
count,100.000000,100.000000,100.000000,100.000000,100.000000
mean,106.840000,5.110000,28.406643,12.670000,19.033327
std,49.244744,2.029554,30.871132,8.012686,32.077671
min,18.000000,1.000000,6.333333,1.000000,2.842105
25%,73.000000,4.000000,14.250000,5.000000,5.864286
50%,103.000000,6.000000,18.309524,13.500000,8.410526
75%,143.500000,7.000000,25.071429,18.000000,12.975000
max,276.000000,7.000000,174.000000,34.000000,174.000000


In [10]:
from util.clustering_eval import print_clustering_classes

print_clustering_classes(clustering_prediction, clustering.n_clusters, test_data, message_details=True)


________________ Class 0 ________________

___
1) 
Removed test for deprecated NegationFilter

- - - 
Committer: ingo@silverstripe.com
Project:   silverstripe_silverstripe-framework
___
2) 
BUG Check for instanceof SS_List rather than deprecated DataObjectSet

- - - 
Committer: ingo@silverstripe.com
Project:   silverstripe_silverstripe-fulltextsearch
___
3) 
Mark private static deprecation notice for <I>

Its too intrusive for <I> at the moment

- - - 
Committer: ingo@silverstripe.com
Project:   silverstripe_silverstripe-framework
___
4) 
MINOR Temporarily disabled deprecation notice in DataObjectDecorator and LeftAndMainDecorator, to be re-enabled once we're in beta stage (they unnecessarily break too many modules for now

- - - 
Committer: ingo@silverstripe.com
Project:   silverstripe_silverstripe-framework
___
5) 
Allow for short versions in Deprecation::notification_version()

- - - 
Committer: ingo@silverstripe.com
Project:   silverstripe_silverstripe-framework
___
6) 
Increased 